In [1]:
import sqlite3
import operator
import random
from functools import reduce
from pyvis.network import Network
from itertools import accumulate
from math import floor
from multiprocessing import Pool

In [2]:
# 圖片高度
pxHeight='800px'
# 圖片寬度
ratioWidth='100%'
# 背景顏色
colorbBg='#222222'
# 字體顏色
colorNodeFont='white'

# 預計開多少行程處理
iProcNum = 3
# 把感染的總工作量切成幾個 chunks(看是要切成行程數量的幾倍)
iChunksNum = iProcNum + 1
# 每一個行程最多可以等待的秒數
iWaitWkrSec = 86400
# 打疫苗之後的感染性
floatVaccInfProb = 0.0
# 被感染的節點顏色
hexInfColor = '#fc0303'
# 打過疫苗的節點顏色
hexVaccColor = '#7cff0a'
# 群體中想要接種的比例
floatVaccRatio = 0.2

# 從疫情開始到第一天打疫苗中間的天數，簡單的說就是疫情前面沒有打疫苗的天數
iHeadDaysNoVacc = 0
# 上面那個比例要分幾天打完（這個數字要 <= iDays）
iVaccDays = 10
# 打完疫苗以後剩下沒疫苗打的天數
iTailDaysNoVacc = 0

# 第一個被感染的人
iFirstInfPid = 0

# 底下是用在 random.uniform(low, up)
# 最大可能接觸人數隨機微調係數下限值
floatInfRandFactorLow = 0.9
# 最大可能接觸人數隨機微調係數上限值
floatInfRandFactorUpper = 1.0

# 最大可能接觸人數(隔離政策從這裡調整)
tupleMaxConn = (1, 3, 9, 49)
# 最大可能接觸人數的權重（越大越可能被挑出來）
tupleWtOfChoiceMaxConn = (2, 11, 13, 3)
# 依感染天數決定感染力的字典
dictInfectivity = {1: 0, 2: 0, 3: 0, 4: 0.6, 5: 0.7, 6: 0.6, 7: 0.4, 8: 0.2, 9: 0.1, 10: 0}
# dictInfectivity = {1: 0, 2: 0, 3: 0, 4: 0.1, 5: 0.1, 6: 0.1, 7: 0.1, 8: 0.1, 9: 0.1, 10: 0}

# 族群母數
iPopCnt = 500

In [3]:
# parameter check
# 先檢查總人數，如果小於等於 max(tupleMaxConn)，
# 例如最大人數 49 ， max(tupleMaxConn) 也是 49 ，這樣是不行的
# 因為總人數 49 ，剩下有可能接觸到最大人數就是 48
assert(max(tupleMaxConn) < iPopCnt), '總人數扣除自己以後要 >= tupleMaxConn 的最大元素。'

In [4]:
class Person:
    def __init__(self, iPid, iInfDays=0, boolInfFlag=False, floatInfProb=0.0, boolVaccFlag=False, ):
        self.pid = iPid
        self.boolIfInf = boolInfFlag
        self.boolIfVacc = boolVaccFlag
        self.floatInfProbability = floatInfProb
        self.iInfDays = iInfDays
        self.iMaxConnCount = random.choices(tupleMaxConn, tupleWtOfChoiceMaxConn)[0]
        # 建立好所有人與人關係之後才填入
        self.listMayContactPids = []

    def IsInfected(self):
        self.boolIfInf = True

    def IsVacced(self):
        self.boolIfVacc = True
        self.floatInfProbability = floatVaccInfProb

    def ResetInfStatus(self):
        self.boolIfInf = False
        self.boolIfVacc = False
        self.floatInfProbability = 0.0
        self.iInfDays = 0

In [5]:
# 找出跟這個人有關的所有關係人
# (除了他跟別人建立關係，別人也會跟他建立關係，這裡會找出雙向關係然後去重複)
def getRelatePidsWithThisMan(iPid, listAllPopRels):
    listRelsWithThisPerson = list(filter(lambda x: x[0] == iPid or x[1] == iPid, listAllPopRels))
    if listRelsWithThisPerson == []:
        return []
    # 挑出跟這個人有關的人際關係
    listGrpMightBeInf = list(set(reduce(lambda x, y: x + y, listRelsWithThisPerson)))
    # 因為是用 reduce 產生 listGrpMightBeInf，所以底下要移除這個 iPid
    listGrpMightBeInf.remove(iPid)
    return listGrpMightBeInf


def NetworkMaterialization(listAllPop, listAllPopRels, sNetworkName):
    netPRInf = Network(height=pxHeight, width=ratioWidth, bgcolor=colorbBg, font_color=colorNodeFont)
    for person in listAllPop:
        if person.boolIfInf == True:
            netPRInf.add_node(person.pid, title=str(person.pid), color=hexInfColor, value=10)
        elif person.boolIfVacc == True:
            netPRInf.add_node(person.pid, title=str(person.pid), color=hexVaccColor, value=10)
        else:
            netPRInf.add_node(person.pid, title=str(person.pid), value=10)

    for relation in listAllPopRels:
        netPRInf.add_edge(relation[0], relation[1])

    netPRInf.barnes_hut()
    netPRInf.show(f'{sNetworkName}.html')


# 把要接種疫苗的總人數依要接種的天數分批
def getListChunkNums(iToBeDivided, iChunkCnt):
    if iChunkCnt == 0:
        return []
    iStep = floor(iToBeDivided / iChunkCnt)
    listChunks = [iStep] * iChunkCnt
    listChunks[-1] += iToBeDivided - sum(listChunks)
    return sorted(listChunks, reverse=True)


def getlistChunkPids(iTotal, iDivideGrpNo):
    listChunkCutNums = list(accumulate([0] + getListChunkNums(iTotal, iDivideGrpNo), operator.add))
    listTmpA = listChunkCutNums[0:-1]
    listTmpB = listChunkCutNums[1:]
    listTupleChunkCut = list(zip(listTmpA, listTmpB))
    listReturn = []
    for tupleChunkCut in listTupleChunkCut:
        listChunkPids = [pid for pid in range(tupleChunkCut[0], tupleChunkCut[1])]
        listReturn.append(listChunkPids)
    return listReturn


# （接種策略）找出接觸最多
#       1. 沒打過疫苗(才有傳染可能)
#       2. 沒有被感染(才能打疫苗)
#   的人群的人
def getListTopNGrpByContactCntNoVacc(listAllPop, iTopN=100, boolSortContactCntReverse=True):
    listPidWithContactCnt = []
    # 取出沒有打過疫苗且沒有被感染的 pids ，因為這些人是要打疫苗的
    # 但是因為感染的太快，所以有可能會沒辦法打到想要的人數比例。
    listAllNotVaccNotInfPids = list(
        map(lambda x: x.pid, filter(lambda x: x.boolIfVacc is False and x.boolIfInf is False, listAllPop)))
    # 取出打過疫苗的 pids
    listAllVaccPids = list(map(lambda y: y.pid, filter(lambda x: x.boolIfVacc is True, listAllPop)))

    # 這裡計算排列依據的接觸人數有個重點，
    # （策略）要把會接觸到的人中挖掉打過疫苗的人
    # 但是不能挖掉被感染的人，因為這種人會讓接種候選者感染機會增加，
    # 而這些人又剛好是接觸人群比較多的人，所以不能挖掉。
    for intPid in listAllNotVaccNotInfPids:
        # 把會接觸到的人中挖掉打過疫苗的人
        listMightBeInfPids = list(filter(lambda x: x not in listAllVaccPids, listAllPop[intPid].listMayContactPids))
        listPidWithContactCnt.append((intPid, len(listMightBeInfPids)))
    # 0 號不能打疫苗，是第一位感染者。
    listPidWithContactCnt = list(filter(lambda x: x[0] != 0, listPidWithContactCnt))
    listTopNPidsByContactCnt = sorted(listPidWithContactCnt, key=lambda x: x[1], reverse=boolSortContactCntReverse)[
                               0:iTopN]
    return listTopNPidsByContactCnt


# 一開始決定人與人關係的函數 (以這個人為出發點單向的和別人建立關係)
# 在這裡建立人與人關係的時候會去掉自己跟自己建立關係
def GetListPopRels(listPop):
    listPopRels = []
    for person in listPop:
        # 把 person 挑掉
        listPeopleWithoutThisMan = [x for x in listPop if x.pid != person.pid]
        # 根據隨機選出的 MaxConn 數目挑出可能接觸的 pid
        listPopConnTo = random.sample(listPeopleWithoutThisMan, k=person.iMaxConnCount)

        for PersonConnTo in listPopConnTo:
            if person.pid < PersonConnTo.pid:
                tuplePR = (person.pid, PersonConnTo.pid)
            else:
                tuplePR = (PersonConnTo.pid, person.pid)
            listPopRels.append(tuplePR)
    listPopRels = list(set(listPopRels))
    return listPopRels


def wkrDealSubInfNet(chunk_Pids, dict_InfOddsByDay, float_InfOddsModFactorLow, float_InfOddsModFactorUpper, list_Pop,
                     list_VaccPids):
    listForReturn = []
    for iPid in chunk_Pids:
        person = list_Pop[iPid]
        if person.boolIfVacc == True:
            continue
        # 從關係者清單中挖掉打過疫苗的人，剩下的就是可以感染的人
        listMightBeInfPidsNoVacc = list(filter(lambda x: x not in list_VaccPids, person.listMayContactPids))

        # 如果被感染且感染天數是 0 就下一個 iteration
        # 在換天的時候，會幫每個已經受感染的人天數加 1
        if person.boolIfInf is True and person.iInfDays == 0:
            continue

        # 首先依照被感染天數決定感染機率
        person.floatInfProbability = dict_InfOddsByDay.get(person.iInfDays, 0)

        listBeInfPids = []
        # 如果感染機率大於 0 ，就開始感染過程。
        if person.floatInfProbability > 0:
            # # min 的第一個參數是沒有接種可能被感染的人數，會越來越少，所以不加微調參數
            # # 第二個參數是隨機算出的最大接觸人數
            # # 這裡是算出每天會接觸到的可能被感染的大概人數
            # iSampleSize = round(
            #     min(
            #         len(listMightBeInfPidsNoVacc),
            #         person.iMaxConnCount
            #     ) * random.uniform(float_InfOddsModFactorLow, float_InfOddsModFactorUpper)
            # )

            # 這裡是算出每天會接觸到的可能被感染的大概人數
            iSampleSize = round(person.iMaxConnCount * random.uniform(float_InfOddsModFactorLow, float_InfOddsModFactorUpper))
            # 取出這些人
            assert(iSampleSize <= len(person.listMayContactPids)), 'iSampleSize <= len(person.listMayContactPids is necessary'
            listBeContactedPids = random.sample(person.listMayContactPids, k=iSampleSize)
            for i_pid in listBeContactedPids:
                if i_pid not in listMightBeInfPidsNoVacc:
                    continue
                else:
                    boolTmpIfBeInf = random.choices([True, False], [person.floatInfProbability, 1-person.floatInfProbability], k=1)[0]
                    if boolTmpIfBeInf is True:
                        listBeInfPids.append(i_pid)
        listForReturn += listBeInfPids
    return listForReturn


def InfSimu(listPop, listPopRels, floatVaccRrac=floatVaccRatio,
            iNoVaccHeadDays=iHeadDaysNoVacc, iVaccDividedays=iVaccDays, iNoVaccTailDays=iTailDaysNoVacc,
            i1stInfPid=iFirstInfPid, boolIfSortContactCntReverse=True,
            dictInfOddsByDay=dictInfectivity,
            floatInfOddsModFactorLow=floatInfRandFactorLow, floatInfOddsModFactorUpper=floatInfRandFactorUpper):
    # 看需要多少接種接種比例，用這個比例算出在 iVaccDays 內每天要接種多少人的 list
    # 例如有 10 人要分 3 天接種，則得到 [3, 3, 4]
    # 這裡要注意，總人數乘以浮點數會變成浮點數，所以分批完以後會變成浮點數。
    # 而這個數字是要拿來做 slice 的 index ，這樣會出錯。
    # 所以要進行數字分批的時候要注意，輸入 getListChunks 的參數必須是整數（要 round 過）。
    iObservAfterdays = iNoVaccHeadDays + iVaccDividedays + iNoVaccTailDays
    listVaccChunkCnts = [0] * iNoVaccHeadDays + getListChunkNums(round(len(listPop) * floatVaccRrac),
                                                                 iVaccDividedays) + [0] * iNoVaccTailDays

    # 重設 listPeople
    for person in listPop:
        person.ResetInfStatus()
    # 感染第一個人
    listPop[i1stInfPid].IsInfected()

    # 多行程處理感染
    with Pool(iProcNum) as poolProc:
        for day in range(iObservAfterdays):
            # 先幫被感染的人天數加 1
            for p in listPop:
                if p.boolIfInf is True:
                    p.iInfDays += 1

            # 在每一天開始的時候從 listVaccChunkCnts 中挑一個數字出來決定要幫多少人打疫苗
            iVaccChunk = listVaccChunkCnts.pop(0) if len(listVaccChunkCnts) > 0 else 0

            # 每天依照接觸不同人數作排序，挑出要打疫苗的 (pids, ContactCnt)
            # (因為每天打疫苗都是隨機變動的，
            # 所以雖然我知道哪些人會接觸最多人是固定的，
            # 但是扣掉打疫苗之後就是會變動的，所以要每天重排。)
            listTopNPidsWithContactCntNoVacc = getListTopNGrpByContactCntNoVacc(listPop, iTopN=iVaccChunk,
                                                                                boolSortContactCntReverse=boolIfSortContactCntReverse)
            # 分離要打疫苗族群的 Pids
            listPidsForVacc = list(map(lambda x: x[0], listTopNPidsWithContactCntNoVacc))
            # 每天的打疫苗步驟
            for PidForVac in listPidsForVacc:
                listPop[PidForVac].boolIfVacc = True

            # 每天建立有打疫苗的 pids list
            listPidsVacc = list(map(lambda y: y.pid, filter(lambda x: x.boolIfVacc is True, listPop)))

            # 依照多行程的行程數目決定要把全部人分成幾個 chunks
            listInfNetChunks = getlistChunkPids(len(listPop), iChunksNum)
            # 設定非同步執行多行程找出今天被感染的 pids
            listTmpInfResult = [poolProc.apply_async(wkrDealSubInfNet, (chunkPids, dictInfOddsByDay, floatInfOddsModFactorLow, floatInfOddsModFactorUpper, listPop,
                                                                        listPidsVacc)) for chunkPids in listInfNetChunks]
            # 把每個行程中今天被感染的 pids 取回來
            listInfResultChunks = [res.get(timeout=iWaitWkrSec) for res in listTmpInfResult]

            # 把取回來的每個子 list 合併成一個大的(去重複並且排序)
            listInfThisDayPids = sorted(list(set(reduce(lambda acc, x: acc + x, listInfResultChunks))))
            if listInfThisDayPids == []:
                continue
            for iPid in listInfThisDayPids:
                listPop[iPid].IsInfected()

    return listPop, listPopRels


def wkrGetRelationsForPid(listChunkPids, listAllPopRels):
    listPidWithCintacts = []
    for iPid in listChunkPids:
        list_MayContactPids = getRelatePidsWithThisMan(iPid, listAllPopRels)
        listPidWithCintacts.append((iPid, list_MayContactPids))
    return listPidWithCintacts

In [6]:
connPR = sqlite3.connect('./PeopleRelation.db')
cur = connPR.cursor()

In [7]:
# drop 所有表格
cur.execute('drop table if exists Relation')
cur.execute('drop table if exists People')

In [8]:
cur.execute(
'''
    create table if not exists People( pid integer not null primary key )
''')

cur.execute(
'''
    create table if not exists Relation( pid integer not null,
    pidconn integer not null,
    primary key(pid,
    pidconn),
    foreign key (pid) references People(pid),
    foreign key (pidconn) references People(pid) )
''')

In [9]:
# Jupyter 中不用
# if __name__ == '__main__':
listPeople = [Person(x) for x in range(iPopCnt)]

In [10]:
# insert data into table People
for person in listPeople:
    cur.execute(f'insert into People values({person.pid})')
connPR.commit()

In [11]:
# 模擬人與人之間的接觸關係
listPR = GetListPopRels(listPeople)

# 幫每個人填入他有機會遇到的所有 pids
listChunksForMp = getlistChunkPids(len(listPeople), iProcNum)

with Pool(processes=iProcNum) as pool:
    listTuplePidWithContacts = [pool.apply_async(wkrGetRelationsForPid, (listChunkPids, listPR)) for listChunkPids in listChunksForMp]
    listChunkResults = [res.get(timeout=iWaitWkrSec) for res in listTuplePidWithContacts]
listTuplePidWithContacts = reduce(lambda acc, x: acc + x, listChunkResults)
for tuplePidWithContacts in listTuplePidWithContacts:
    listPeople[tuplePidWithContacts[0]].listMayContactPids = tuplePidWithContacts[1]

In [12]:
cur.executemany('insert into Relation values(?, ?)', listPR)
connPR.commit()

In [13]:
# 畫出未受感染的群體
NetworkMaterialization(listPeople, listPR, 'NetOrig')

In [14]:
# 未施予疫苗保護的群體
listPeople, listPR = InfSimu(listPeople, listPR, floatVaccRrac=0)
NetworkMaterialization(listPeople, listPR, 'NetOrigInf')

In [15]:
len(list(filter(lambda x: x.boolIfInf == False, listPeople)))

124

In [16]:
# 以接觸最少人數選擇打疫苗族群的策略
listPeople, listPR = InfSimu(listPeople, listPR, boolIfSortContactCntReverse=False)
NetworkMaterialization(listPeople, listPR, 'NetVaccByContactCntAsc')

In [17]:
len(list(filter(lambda x: x.boolIfInf == False, listPeople)))

161

In [18]:
# 以接觸最多人數選擇打疫苗族群的策略
listPeople, listPR = InfSimu(listPeople, listPR)           
NetworkMaterialization(listPeople, listPR, 'NetVaccByContactCntDesc')

In [19]:
len(list(filter(lambda x: x.boolIfInf == False, listPeople)))

316

In [20]:
connPR.close()